In [3]:
# Importing necessary libraries
from abc import ABC, abstractmethod  # Abstract base class and method
import torch  # PyTorch for deep learning models
import torch.nn as nn  # Neural network modules from PyTorch
import torch.optim as optim  # Optimizers for training models
import torch.nn.functional as F  # Functions for layers and activations
from sklearn.ensemble import RandomForestClassifier  # Random Forest classifier from sklearn
import joblib  # For saving and loading models
import torchvision  # For MNIST dataset and other torchvision models
import torchvision.transforms as transforms  # For data transformations
from torch.utils.data import DataLoader  # For batching data
from tqdm import tqdm  # Progress bar for loops
import matplotlib.pyplot as plt  # For visualizations

# Taking user input for the algorithm selection
use_algorithm = input(str('Input one algorithm as sample: cnn, nn, rf'))

# Interface class for all models, ensuring that models implement `train_model` and `predict`
class MnistClassifierInterface:
    @abstractmethod
    def train_model(self, train_loader, val_loader, epochs=10):
        raise NotImplementedError
        
    @abstractmethod
    def predict(self, X):
        raise NotImplementedError


# --- Feed-Forward Neural Network Class ---
class FeedForwardNN(nn.Module, MnistClassifierInterface):
    def __init__(self, input_size=28*28, hidden_size=128, output_size=10):
        # Initialize the neural network with input, hidden and output sizes
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),  # Input layer
            nn.ReLU(),  # ReLU activation
            nn.Linear(hidden_size, output_size)  # Output layer (10 classes for MNIST)
        )

    def forward(self, x):
        return self.model(x)

    def train_model(self, train_loader, val_loader, epochs=10, save_path="ffnn_best.pth"):
        # Method to train the feed-forward neural network
        optimizer = optim.Adam(self.parameters(), lr=0.001)  # Adam optimizer
        criterion = nn.CrossEntropyLoss()  # Loss function for classification
        best_loss = float("inf")

        for epoch in tqdm(range(epochs)):
            self.model.train()
            total_loss = 0
            for images, labels in train_loader:
                images = images.view(images.size(0), -1)  # Flatten images
                optimizer.zero_grad()  # Zero out gradients
                outputs = self(images)  # Get outputs
                loss = criterion(outputs, labels)  # Compute loss
                loss.backward()  # Backpropagate gradients
                optimizer.step()  # Update weights
                total_loss += loss.item()

            val_loss = self.evaluate(val_loader, criterion)  # Validation loss
            print(f"Epoch {epoch+1}: Train Loss={total_loss:.4f}, Val Loss={val_loss:.4f}")

            # Save model if validation loss improves
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(self.state_dict(), save_path)
                print(f"Saved best model parameters - Epoch {epoch+1}, Val Loss: {val_loss:.4f}")

    def evaluate(self, loader, criterion):
        # Method to evaluate model on validation set
        self.eval()
        total_loss = 0
        with torch.no_grad():  # No gradient tracking in evaluation
            for images, labels in loader:
                images = images.view(images.size(0), -1)
                outputs = self(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
        return total_loss / len(loader)

    def predict(self, X):
        # Method to make predictions on new data
        self.eval()
        with torch.no_grad():
            X = X.view(X.size(0), -1)
            outputs = self(X)
            return torch.argmax(outputs, dim=1)


# --- Convolutional Neural Network Class ---
class CNN(nn.Module, MnistClassifierInterface):
    def __init__(self):
        # Initialize CNN with multiple convolutional and pooling layers
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),  # First convolutional layer
            nn.ReLU(),
            nn.MaxPool2d(2),  # Max pooling layer
            nn.Conv2d(16, 32, kernel_size=3, padding=1),  # Second convolutional layer
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),  # Flatten the image for the fully connected layer
            nn.Linear(32 * 7 * 7, 128),  # Fully connected layer
            nn.ReLU(),
            nn.Linear(128, 10)  # Output layer (10 classes for MNIST)
        )

    def forward(self, x):
        return self.model(x)

    def train_model(self, train_loader, val_loader, epochs=10, save_path="cnn_best.pth"):
        # Method to train the CNN model
        optimizer = optim.Adam(self.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        best_loss = float("inf")

        for epoch in tqdm(range(epochs)):
            self.model.train()
            total_loss = 0
            for images, labels in train_loader:
                optimizer.zero_grad()
                outputs = self(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            val_loss = self.evaluate(val_loader, criterion)
            print(f"Epoch {epoch+1}: Train Loss={total_loss:.4f}, Val Loss={val_loss:.4f}")

            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(self.state_dict(), save_path)
                print(f"Saved best model parameters - Epoch {epoch+1}, Val Loss: {val_loss:.4f}")
                
    def evaluate(self, loader, criterion):
        # Method to evaluate the CNN model on validation set
        self.eval()
        total_loss = 0
        with torch.no_grad():
            for images, labels in loader:
                outputs = self(images)
                loss = criterion(outputs, labels)
                total_loss += loss.item()
        return total_loss / len(loader)

    def predict(self, X):
        # Method to make predictions using CNN
        self.eval()
        with torch.no_grad():
            outputs = self(X)
            return torch.argmax(outputs, dim=1)


# --- Random Forest Class for MNIST ---
class RandomForestMnist(MnistClassifierInterface):
    def __init__(self):
        # Initialize Random Forest model from sklearn
        self.model = RandomForestClassifier(n_estimators=100, random_state=42)

    def train_model(self, train_loader, val_loader=None, epochs=10, save_path="rf_best.pkl"):
        # Train the Random Forest model
        X_train, y_train = self._prepare_data(train_loader)
        self.model.fit(X_train, y_train)
        joblib.dump(self.model, save_path)

    def _prepare_data(self, loader):
        # Convert images and labels into a format suitable for Random Forest
        X, y = [], []
        for images, labels in loader:
            images = images.view(images.size(0), -1).numpy()
            X.extend(images)
            y.extend(labels.numpy())
        return X, y

    def predict(self, X):
        # Make predictions using Random Forest
        X = X.view(X.size(0), -1).numpy()
        return self.model.predict(X)


# --- Class to manage training process and select models ---
class MnistClassifier:
    def __init__(self, algorithm):
        # Select the model based on user input
        self.algorithm = algorithm
        if algorithm == "cnn":
            self.model = CNN()
        elif algorithm == "nn":
            self.model = FeedForwardNN()
        elif algorithm == "rf":
            self.model = RandomForestMnist()
        else:
            raise ValueError("Unsupported algorithm. Choose from ['cnn', 'nn', 'rf']")

    def train_model(self, train_loader, val_loader, epochs=10):
        # Train the selected model
        self.model.train_model(train_loader, val_loader, epochs=epochs)

    def predict(self, X):
        # Make predictions using the selected model
        return self.model.predict(X)


# --- Load MNIST Dataset ---
transform = transforms.Compose([transforms.ToTensor()])  # Transform to tensor
train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
val_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)

# DataLoader for batching the dataset
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Select the model based on user input
classifier = MnistClassifier(algorithm=use_algorithm)
classifier.train_model(train_loader, val_loader, epochs=10)

# Make predictions
sample_data, _ = next(iter(val_loader))
predictions = classifier.predict(sample_data)
print(predictions)



# Function to prepare and display an image sample
def show_sample_image(inpt):
    # Load the trained model based on the input algorithm type
    if inpt == "rf":  # Random Forest model
        classifier = MnistClassifier(algorithm="rf")  # Instantiate the RandomForest classifier
        classifier.model = joblib.load("rf_best.pkl")  # Load the pre-trained Random Forest model

    else:
        if inpt == "nn":  # Feed-Forward Neural Network model
            classifier = MnistClassifier(algorithm="nn")  # Instantiate the Feed-Forward Neural Network classifier
            classifier.model.load_state_dict(torch.load("ffnn_best.pth"))  # Load the pre-trained NN model weights
        else:  # Convolutional Neural Network model
            classifier = MnistClassifier(algorithm="cnn")  # Instantiate the CNN classifier
            classifier.model.load_state_dict(torch.load("cnn_best.pth"))  # Load the pre-trained CNN model weights
    
    # Load the test dataset (MNIST)
    transform = transforms.Compose([transforms.ToTensor()])  # Transform the images to tensors
    test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)  # Load the MNIST test dataset
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True)  # Create a DataLoader for the test set with batch size 1
    
    # Get the first image and its label from the test dataset
    sample_images, sample_labels = next(iter(test_loader))  # Extract the first batch of images and labels
    image = sample_images[0].squeeze()  # Remove extra dimensions to display the image properly
    label = sample_labels[0].item()  # Get the label (true class) of the first image

    # Prepare the image for prediction (convert to the appropriate format based on model type)
    if inpt == "rf":  # For Random Forest, convert the image into a 1D vector (flatten the 28x28 image into a 784-length vector)
        img = sample_images[0].view(-1).numpy().reshape(1, -1)  # Flatten and convert to numpy array for Random Forest
    else:  # For NN and CNN, add batch dimension to the image (already in tensor form)
        img = sample_images[0].unsqueeze(0)  # Add batch dimension for the neural networks

    # Perform prediction using the classifier's abstracted prediction method
    prediction = classifier.predict(img)  # Make the prediction
    print(f"Predicted class: {prediction.item()}")  # Print the predicted class

    # Visualize the sample image
    plt.imshow(image, cmap="gray")  # Display the image in grayscale
    plt.title(f"Label: {label}, Predicted: {prediction.item()}")  # Display the true label and predicted label in the title
    plt.axis('off')  # Hide the axis to focus on the image
    plt.show()  # Show the image with the title

# Main block to visualize the image sample based on the chosen model
if __name__ == "__main__":
    show_sample_image(use_algorithm)  # Call the function with the selected algorithm (cnn, nn, or rf)


Input one algorithm as sample: cnn, nn, rf cnn


  0%|                                                                                           | 0/10 [00:05<?, ?it/s]


KeyboardInterrupt: 